In [1]:
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pl_bolts.datamodules import MNISTDataModule
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchmetrics

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('/mnist/train.csv')
arr = df.to_numpy()
X = torch.tensor(arr[:, 1:], dtype=torch.float)

In [8]:
torch.mean(X)/255

tensor(0.1310)

# MNISTDataset

In [2]:
class MNISTDataset(Dataset):
    def __init__(self, data_dir: str, testing: bool):

        self.testing = testing

        data_path = data_dir + ('/test.csv' if testing else '/train.csv')
        df = pd.read_csv(data_path)
        arr = df.to_numpy()

        if testing:
            self.X = torch.tensor(arr, dtype=torch.float)
        else:
            self.X = torch.tensor(arr[:, 1:], dtype=torch.float)
            self.y = torch.tensor(arr[:, :1]).squeeze()
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, index: int):

        if self.testing:
            return self.X[index]
        else:
            return self.X[index], self.y[index]

# MNISTDataModule

In [3]:
class MNISTDataModule(pl.LightningDataModule):

    def __init__(self, data_dir: str, batch_size: int):

        super().__init__()

        self.data_dir = data_dir
        self.batch_size = batch_size

    def setup(self, stage: str):

        if stage == "fit":
            self.mnist_train = MNISTDataset(self.data_dir, testing=False)

        if stage == "predict":
            self.mnist_test = MNISTDataset(self.data_dir, testing=True)

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=self.batch_size)

    def predict_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=self.batch_size)

# Model

In [4]:
class MNISTModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(28 * 28, 10),
        )

    def forward(self, X):
        return self.model(X)

    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=0.01)
        
    def compute_loss_and_accuracy(self, batch):

        X, y = batch
        logits = self(X)
        
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        accuracy = torchmetrics.functional.accuracy(preds, y)

        return loss, accuracy

    def training_step(self, batch, batch_idx):

        loss, accuracy = self.compute_loss_and_accuracy(batch)
        self.log("train_loss", loss)
        self.log("train_acc", accuracy)

        return loss

    def validation_step(self, batch, batch_idx):

        loss, accuracy = self.compute_loss_and_accuracy(batch)
        self.log("val_loss", loss)
        self.log("val_acc", accuracy)

    def predict_step(self, batch, batch_idx):
        
        X = batch
        logits = self(X)
        preds = torch.argmax(logits, dim=1)

        return preds


In [5]:
trainer = pl.Trainer(
    max_epochs=1,
    gpus=-1)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
model = MNISTModel()
dm = MNISTDataModule('/mnist/', 32)

In [7]:
trainer.fit(model, dm)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:133: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 7.9 K 
-------------------------------------
7.9 K     Trainable params
0         Non-trainable params
7.9 K     Total params
0.031     Total estimated model params size (MB)
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0: 100%|██████████| 1313/1313 [00:08<00:00, 160.21it/s, loss=298, v_num=6]  


In [9]:
preds = trainer.predict(model, dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting DataLoader 0: 100%|██████████| 875/875 [00:01<00:00, -332.12it/s]  


In [10]:
preds

[tensor([2, 0, 9, 9, 3, 7, 0, 3, 0, 3, 5, 7, 4, 0, 4, 5, 3, 1, 9, 0, 9, 1, 1, 5,
         7, 4, 2, 7, 7, 7, 7, 5]),
 tensor([4, 2, 6, 2, 5, 5, 1, 5, 7, 7, 4, 9, 8, 7, 8, 8, 6, 7, 6, 8, 8, 3, 8, 2,
         1, 8, 2, 5, 4, 1, 7, 0]),
 tensor([0, 0, 1, 9, 0, 1, 6, 5, 8, 8, 2, 8, 3, 9, 2, 2, 5, 9, 1, 5, 9, 2, 4, 3,
         6, 7, 2, 0, 6, 6, 1, 4]),
 tensor([3, 9, 7, 7, 0, 5, 2, 0, 7, 3, 0, 5, 0, 3, 0, 0, 4, 7, 1, 7, 1, 1, 5, 3,
         3, 7, 2, 1, 6, 3, 8, 7]),
 tensor([8, 4, 5, 5, 6, 0, 0, 0, 3, 1, 3, 0, 4, 3, 4, 5, 5, 8, 7, 7, 2, 8, 4, 3,
         5, 6, 5, 3, 7, 5, 7, 8]),
 tensor([3, 0, 4, 5, 1, 8, 7, 6, 4, 0, 2, 7, 8, 6, 1, 3, 7, 4, 1, 2, 4, 7, 5, 6,
         4, 9, 2, 1, 6, 0, 6, 1]),
 tensor([4, 9, 6, 0, 9, 7, 6, 9, 1, 9, 0, 5, 9, 0, 8, 4, 6, 2, 0, 9, 3, 6, 7, 2,
         1, 6, 3, 4, 2, 3, 1, 2]),
 tensor([2, 8, 4, 6, 1, 0, 0, 4, 9, 1, 7, 2, 2, 8, 8, 6, 8, 6, 2, 5, 5, 5, 4, 5,
         3, 5, 9, 7, 1, 3, 8, 4]),
 tensor([5, 1, 4, 5, 6, 5, 8, 5, 7, 0, 6, 8, 6, 1, 4, 0, 6, 3, 9, 4, 1, 

In [12]:
all_preds = torch.empty(0, dtype=torch.int)

for preds_ in preds:

    all_preds = torch.cat((all_preds, preds_))

In [15]:
np_all_preds = all_preds.numpy()

In [17]:
df_all_preds = pd.DataFrame(np_all_preds)

In [26]:
df_all_preds.to_csv('/mnist/out.csv')

In [24]:
df_all_preds.index = np.arange(1, len(df_all_preds) + 1)
df_all_preds.index.name = 'ImageId'
df_all_preds.columns = ['Label']

In [25]:
df_all_preds

,Label
ImageId,
1,2
2,0
3,9
4,9
5,3
...,...
27996,9
27997,7
27998,3
